# Design Flowsheet for NGCC + SOEC with Steam Integration

This flowsheet example provides an off-design model for integration of an SOEC for hydrogen production with an NGCC.  The NGCC nominally produces 650 MW net, and includes 97% CO2 capture.  A detailed SOEC unit model is used here.

## Import Required Modules

In [ ]:
import os
import math
import numpy as np
import pytest
from IPython.core.display import SVG
import pyomo.environ as pyo
from idaes.core.solvers import use_idaes_solver_configuration_defaults
import idaes
import idaes.core.util.scaling as iscale
from idaes.core.util import model_serializer as ms
import idaes.core.util as iutil
import ngcc_soec_costing ### changed name of costing module
from idaes.models_extra.power_generation.costing.power_plant_capcost import (
    QGESSCosting,
    QGESSCostingData,
)
import ngcc_soec

## Make Output Directories

This notebook can produce a large number of output files.  To make it easier to manage, some subdirectories are used to organize output.  This ensures that the directories exist.

In [ ]:
def make_directory(path):
    """Make a directory if it doesn't exist"""
    try:
        os.mkdir(path)
    except FileExistsError:
        pass
    
make_directory("data")
make_directory("data_pfds")
make_directory("data_tabulated")

## Set Global Solver Options

Setting global solver options applies them to any solver created subsequently, including the ones used for initialization.  The user scaling option disables Ipopt's automatic scaling and allows it to use user provided variable scaling.

In [ ]:
use_idaes_solver_configuration_defaults()
idaes.cfg.ipopt.options.nlp_scaling_method = "user-scaling"
idaes.cfg.ipopt.options.linear_solver = "ma57"
idaes.cfg.ipopt.options.OF_ma57_automatic_scaling = "yes"
idaes.cfg.ipopt.options.tol = 1e-5
idaes.cfg.ipopt.options.max_iter = 100
solver = pyo.SolverFactory("ipopt")

## Create and Initialize the NGCC + SOEC Model

In [ ]:
def scale_after_costing_model_added(m):
    # Scale costing variables

    iscale.set_scaling_factor(m.fs.gas_turbine_aux_load, 1e-3)
    iscale.set_scaling_factor(m.fs.steam_turbine_aux_load, 1e-2)
    iscale.set_scaling_factor(m.fs.misc_aux_loads, 1e-2)
    iscale.set_scaling_factor(m.fs.transformer_losses, 1e-3)
    iscale.set_scaling_factor(m.fs.aux_load, 1e-5)
    iscale.set_scaling_factor(m.fs.fuel_gas_flow[0.0], 1e-5)
    iscale.set_scaling_factor(m.fs.feedwater_flow[0.0], 1e-6)
    iscale.set_scaling_factor(m.fs.hrsg_duty[0.0], 1e-5)
    iscale.set_scaling_factor(m.fs.hrsg_gas_flow[0.0], 1e-5)
    iscale.set_scaling_factor(m.fs.absorber_gas_flow[0.0], 1e-5)
    iscale.set_scaling_factor(m.fs.stack_gas_flow[0.0], 1e-5)
    iscale.set_scaling_factor(m.fs.soec.water_heater02.costing.base_cost_per_unit, 1e-5) 
    iscale.set_scaling_factor(m.fs.soec.water_heater02.costing.capital_cost, 1e-6) 
    iscale.set_scaling_factor(m.fs.soec.cmp01.costing.base_cost_per_unit, 1e-6) 
    iscale.set_scaling_factor(m.fs.soec.cmp01.costing.capital_cost, 1e-6) 
    iscale.set_scaling_factor(m.fs.soec.cmp03.control_volume.deltaP[0.0], 1e-2) 
    iscale.set_scaling_factor(m.fs.soec.cmp04.control_volume.deltaP[0.0], 1e-2) 
    iscale.set_scaling_factor(m.fs.costing.total_TPC, 1e-3) 
    iscale.set_scaling_factor(m.fs.b3.costing.total_plant_cost['6.1'], 1e-1) 
    iscale.set_scaling_factor(m.fs.b5a.costing.bare_erected_cost['5.1.a.epri'], 1e-1) 
    iscale.set_scaling_factor(m.fs.b5a.costing.total_plant_cost['5.1.a.epri'], 1e-1) 
    iscale.set_scaling_factor(m.fs.b6.costing.bare_erected_cost['5.1.b'], 1e-1)
    iscale.set_scaling_factor(m.fs.b6.costing.total_plant_cost['5.1.b'], 1e-1)
    iscale.set_scaling_factor(m.fs.soec.water_heater01.costing.base_cost_per_unit, 1e-5) 
    iscale.set_scaling_factor(m.fs.soec.water_heater01.costing.capital_cost, 1e-6)
    iscale.set_scaling_factor(m.fs.ngcc.st.cond_pump.ratioP[0.0], 1e-2)
    iscale.set_scaling_factor(m.fs.soec.soec_module.number_cells, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.soec_module.costing.base_cost_per_unit, 1e-5)
    iscale.set_scaling_factor(m.fs.soec.soec_module.costing.capital_cost, 1e-5) 
    iscale.set_scaling_factor(m.fs.soec.soec_module.costing.weight, 1e-4)
    iscale.set_scaling_factor(m.fs.soec.soec_module.costing.base_cost_per_unit, 1e-4) 
    iscale.set_scaling_factor(m.fs.soec.soec_module.costing.capital_cost, 1e-4)
    iscale.set_scaling_factor(m.fs.soec.soec_module.costing.weight, 1e-5) 
    iscale.set_scaling_factor(m.fs.soec.sweep_hx.costing.base_cost_per_unit, 1e-5)
    iscale.set_scaling_factor(m.fs.soec.sweep_hx.costing.capital_cost, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.sweep_compressor.costing.base_cost_per_unit, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.sweep_compressor.costing.capital_cost, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.sweep_turbine.costing.capital_cost, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.feed_hx01.costing.base_cost_per_unit, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.feed_hx01.costing.capital_cost, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.feed_heater.costing.base_cost_per_unit, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.feed_heater.costing.capital_cost, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.sweep_heater.costing.base_cost_per_unit, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.sweep_heater.costing.capital_cost, 1e-6)
    iscale.set_scaling_factor(m.fs.soec.cmp03.control_volume.deltaP[0.0], 1e-4)
    iscale.set_scaling_factor(m.fs.soec.cmp04.control_volume.deltaP[0.0], 1e-4)
    
    iscale.constraint_scaling_transform(m.fs.ngcc.gt.inject_translator.zero_flow_eqn[0.0,"CH4"], 1e-3)
    iscale.constraint_scaling_transform(m.fs.ngcc.hrsg.sh_ip2.deltaP_tube_uturn_eqn[0.0], 1e-3)
    iscale.constraint_scaling_transform(m.fs.ngcc.hrsg.sh_ip3.deltaP_tube_uturn_eqn[0.0], 1e-3)
    iscale.constraint_scaling_transform(m.fs.ngcc.st.reboiler.reboiler_condense_eqn[0.0], 1e-3)
    iscale.constraint_scaling_transform(m.fs.soec.soec_module.costing.weight_eq, 1e-3)
    iscale.constraint_scaling_transform(m.fs.soec.sweep_compressor.costing.base_cost_per_unit_eq, 1e-5)
    iscale.constraint_scaling_transform(m.fs.soec.sweep_hx.costing.base_cost_per_unit_eq, 1e-5)
    iscale.constraint_scaling_transform(m.fs.soec.sweep_turbine.costing.capital_cost_constraint, 1e-4)
    iscale.constraint_scaling_transform(m.fs.soec.feed_hx01.costing.base_cost_per_unit_eq, 1e-5)
    iscale.constraint_scaling_transform(m.fs.soec.feed_heater.costing.base_cost_per_unit_eq, 1e-4)
    iscale.constraint_scaling_transform(m.fs.soec.feed_heater.costing.capital_cost_constraint, 1e-6)
    iscale.constraint_scaling_transform(m.fs.soec.sweep_heater.costing.base_cost_per_unit_eq, 1e-4)
    iscale.constraint_scaling_transform(m.fs.soec.sweep_heater.costing.capital_cost_constraint, 1e-6)
    iscale.constraint_scaling_transform(m.fs.soec.water_heater01.costing.base_cost_per_unit_eq, 1e-5)
    iscale.constraint_scaling_transform(m.fs.soec.water_heater01.costing.capital_cost_constraint, 1e-5)
    iscale.constraint_scaling_transform(m.fs.soec.water_heater02.costing.base_cost_per_unit_eq, 1e-5)
    iscale.constraint_scaling_transform(m.fs.soec.water_heater02.costing.capital_cost_constraint, 1e-5)
    iscale.constraint_scaling_transform(m.fs.soec.cmp01.costing.base_cost_per_unit_eq, 1e-5)
    iscale.constraint_scaling_transform(m.fs.sweep_pressure_eqn[0.0], 1e-5)
    iscale.constraint_scaling_transform(m.fs.soec.makeup_mix.mixer1_pressure_eqn[0.0], 1e-6)
    iscale.constraint_scaling_transform(m.fs.ngcc.hrsg.sh_ip3.deltaP_tube_uturn_eqn[0.0], 1e-3)
    iscale.constraint_scaling_transform(m.fs.soec.sweep_hx.costing.capital_cost_constraint, 1e-6)
    iscale.constraint_scaling_transform(m.fs.soec.feed_hx01.costing.capital_cost_constraint, 1e-6)
    iscale.constraint_scaling_transform(m.fs.soec.soec_module.costing.base_cost_constraint, 1e-5)
    iscale.constraint_scaling_transform(m.fs.costing.total_TPC_eq, 1e-3)
    iscale.constraint_scaling_transform(m.fs.soec.soec_module.costing.capital_cost_constraint, 1e-4)
    iscale.constraint_scaling_transform(m.fs.soec.cmp01.costing.capital_cost_constraint, 1e-6)
    iscale.constraint_scaling_transform(m.fs.soec.sweep_compressor.costing.capital_cost_constraint, 1e-6)

if os.path.exists("NGCC_flowsheet_solution.json.gz"):
    # create the ngcc model
    m = pyo.ConcreteModel()
    m.fs = ngcc_soec.NgccSoecFlowsheet(dynamic=False)
    iscale.calculate_scaling_factors(m)
    m.fs.initialize(
        load_from="ngcc_soec_init.json.gz",
    save_to="ngcc_soec_init.json.gz")
    m.fs.ngcc.fuel_cost.fix(4.42)
    m.fs.ngcc.cap_specific_reboiler_duty.fix(2.4e6)
    m.fs.ngcc.cap_fraction.fix(0.97)

    # add capital costing
    m.fs.costing = QGESSCosting()
    ngcc_soec_costing.add_results_for_costing(m)
    ngcc_soec_costing.get_ngcc_soec_capital_cost(m, CE_index_year="2018")
    scale_after_costing_model_added(m)
    # load results from json
    print("Loading prior solved results")
    ms.from_json(m, fname="NGCC_flowsheet_solution.json.gz")
else:
    # create the ngcc model
    m = pyo.ConcreteModel()
    m.fs = ngcc_soec.NgccSoecFlowsheet(dynamic=False)
    iscale.calculate_scaling_factors(m)
    m.fs.initialize(
        load_from="ngcc_soec_init.json.gz",
        save_to="ngcc_soec_init.json.gz")
    print("Solve initial problem")
    res = solver.solve(m, tee=True)
    print("Fix fuel cost and resolve")
    m.fs.ngcc.fuel_cost.fix(4.42)
    res = solver.solve(m, tee=True)
    print("Fix reboiler duty and resolve")
    m.fs.ngcc.cap_specific_reboiler_duty.fix(2.4e6)
    res = solver.solve(m, tee=True)
    print("Fix capture fraction and resolve")
    m.fs.ngcc.cap_fraction.fix(0.97)
    res = solver.solve(m, tee=True)

    # add capital costing
    print("Add initial costing and resolve")
    m.fs.costing = QGESSCosting()
    ngcc_soec_costing.add_results_for_costing(m)        
    res = solver.solve(m, tee=True)
    print("Add capital costing and resolve")
    ngcc_soec_costing.get_ngcc_soec_capital_cost(m, CE_index_year="2018")
    scale_after_costing_model_added(m)
    
    res = solver.solve(m, tee=True)
    print("Saving results to json")
    ms.to_json(m, fname="NGCC_flowsheet_solution.json.gz")

In [ ]:
def display_pfd():
    print("\n\nGas Turbine Section\n")
    display(SVG(m.fs.ngcc.gt.write_pfd()))
    print("\n\nHRSG Section\n")
    display(SVG(m.fs.ngcc.hrsg.write_pfd()))
    print("\n\nSteam Turbine Section\n")
    display(SVG(m.fs.ngcc.st.write_pfd()))
    print("\n\nSOEC Section\n")
    display(SVG(m.fs.soec.write_pfd()))

In [ ]:
m.fs.ngcc.net_power_mw.display()
m.fs.ngcc.st.steam_turbine.throttle_valve[1].deltaP.display()

In [ ]:
decision_vars = []
def make_decision_var(v, lb, ub):
    v.unfix()
    v.setlb(lb)
    v.setub(ub)
    decision_vars.append(v)

# Add constraints for optimization
m.fs.ngcc.st.steam_turbine.throttle_valve[1].deltaP.setub(-1e5)
m.fs.soec.sweep_recycle_split.mixed_state[0].mole_frac_comp["O2"].setub(0.35)
m.fs.soec.feed_recycle_split.mixed_state[0].mole_frac_comp["H2O"].setlb(0.20)
m.fs.soec.sweep_recycle_split.mixed_state[0].temperature.setub(1030)
m.fs.soec.feed_recycle_split.mixed_state[0].temperature.setub(1030)
m.fs.ngcc.st.steam_turbine_lp_split.mixed_state[0].pressure.setlb(2.9e5)

# Design optimization will remove trim heaters, but they are required for dynamic
# operation, so we force in 8 MW capacity heaters.  These bounds allow them to be
# used since they exist. 
m.fs.soec.sweep_heater.control_volume.heat.setlb(0.0)
m.fs.soec.feed_heater.control_volume.heat.setlb(0.0)
m.fs.soec.sweep_heater.control_volume.heat.setub(8.0e6)
m.fs.soec.feed_heater.control_volume.heat.setub(8.0e6)

@m.fs.Constraint(m.fs.time)
def makeup_water_constraint(b, t):
    return m.fs.soec.water_pump.inlet.flow_mol[t] == m.fs.soec.feed_hx01.tube_inlet.flow_mol[t]

# make sure the delta T on the hydrogen side is 75K or less (it's 
# squared so one constraint covers both positive and negative delta T)
@m.fs.Constraint(m.fs.time)
def delta_T_h_constraint(b, t):
    return (
        m.fs.soec.feed_heater.control_volume.properties_out[t].temperature -
        m.fs.soec.feed_recycle_split.mixed_state[t].temperature
    )**2/100 <= 56.25

# make sure the delta T on the oxygen side is 75K or less
@m.fs.Constraint(m.fs.time)
def delta_T_o_constraint(b, t):
    return (
        m.fs.soec.sweep_heater.control_volume.properties_out[t].temperature -
        m.fs.soec.sweep_recycle_split.mixed_state[t].temperature
    )**2/100 <= 56.25

# make sure the oxygen inlet and hydrogen outlet are 75K or less apart
@m.fs.Constraint(m.fs.time)
def delta_T_1_constraint(b, t):
    return (
        m.fs.soec.sweep_heater.control_volume.properties_out[t].temperature -
        m.fs.soec.feed_recycle_split.mixed_state[t].temperature
    )**2/100 <= 56.25

# make sure the hydrogen inlet and oxygen outlet are 75K or less apart
@m.fs.Constraint(m.fs.time)
def delta_T_2_constraint(b, t):
    return (
        m.fs.soec.feed_heater.control_volume.properties_out[t].temperature -
        m.fs.soec.sweep_recycle_split.mixed_state[t].temperature
    )**2/100 <= 56.25

@m.fs.Constraint(m.fs.time)
def average_current_density_constraint(b, t):
    return m.fs.soec.soec_module.solid_oxide_cell.average_current_density[0]/1000 >= -8

m.fs.soec.water_pump.inlet.flow_mol.unfix()
make_decision_var(m.fs.soec.sweep_recycle_split.split_fraction[0, "out"], 0.40, 0.95)
make_decision_var(m.fs.soec.feed_recycle_split.split_fraction[0, "out"], 0.25, 0.95) 
make_decision_var(m.fs.soec.water_split.split_fraction[0, "outlet1"], 0.3, 0.7)
make_decision_var(m.fs.soec.sweep_compressor.inlet.flow_mol[0], 100, 8000)
make_decision_var(m.fs.soec.soec_module.potential_cell[0], 1.26, 1.50)
make_decision_var(m.fs.soec.feed_heater.control_volume.properties_out[0].temperature, 900, 1020)
make_decision_var(m.fs.soec.sweep_heater.control_volume.properties_out[0].temperature, 900, 1020)
make_decision_var(m.fs.soec.soec_module.number_cells, 1e6, 2e6)
make_decision_var(m.fs.soec.sweep_hx.area, 2000, 7000)
make_decision_var(m.fs.soec.feed_hx01.area, 2000, 7000)
make_decision_var(m.fs.soec.water_heater01.area, 2000, 7000)
make_decision_var(m.fs.soec.water_heater02.area, 2000, 7000)

m.fs.obj_design = pyo.Objective(
    expr=(
        m.fs.ngcc.total_variable_cost_rate[0] + 
        m.fs.soec.variable_makeup_water_cost[0] +
        (m.fs.ngcc.net_power[0] + m.fs.soec.total_electric_power[0])/1e6 * 100 + # power out is negative
        m.fs.costing.total_TPC*1e6 * 1.093 * 0.0707 * 1.341 / 365 / 24 # anualized cap. + fixed O&M for SOEC part
    )/1e4
)

iscale.constraint_scaling_transform(m.fs.makeup_water_constraint[0.0], 1e-4)

In [ ]:
# add temperature gradiaent constraints.

def _make_temperature_gradient_terms(fs):
    soec = fs.soec_module.solid_oxide_cell
    dz = soec.zfaces.at(2) - soec.zfaces.at(1)
    # Going to assume that the zfaces are evenly spaced
    for iz in soec.iznodes:
        assert abs(soec.zfaces.at(iz + 1) - soec.zfaces.at(iz) - dz) < 1e-8
    dz = dz * soec.length_z
    def finite_difference(expr, t, ix, iz):
        # Since this is mostly for reference, no need to worry about upwinding or whatever
        if iz == soec.iznodes.first():
            if ix is None:
                return (-1.5 * expr[t, iz] + 2 * expr[t, iz + 1] - 0.5 * expr[t, iz + 2]) / dz
            else:
                return (-1.5 * expr[t, ix, iz] + 2 * expr[t, ix, iz + 1] - 0.5 * expr[t, ix, iz + 2]) / dz
        elif iz == soec.iznodes.last():
            if ix is None:
                return (1.5 * expr[t, iz] - 2 * expr[t, iz - 1] + 0.5 * expr[t, iz - 2]) / dz
            else:
                return (1.5 * expr[t, ix, iz] - 2 * expr[t, ix, iz - 1] + 0.5 * expr[t, ix, iz - 2]) / dz
        else:
            if ix is None:
                return (0.5 * expr[t, iz + 1] - 0.5 * expr[t, iz - 1]) / dz
            else:
                return (0.5 * expr[t, ix, iz + 1] - 0.5 * expr[t, ix, iz - 1]) / dz

    soec.dtemperature_z_dz = pyo.Var(fs.time, soec.iznodes, initialize=0, units=pyo.units.K / pyo.units.m)

    @soec.Constraint(fs.time, soec.iznodes)
    def dtemperature_z_dz_eqn(b, t, iz):
        return b.dtemperature_z_dz[t, iz] == finite_difference(b.temperature_z, t, None, iz)

    soec.fuel_electrode.dtemperature_dz = pyo.Var(
        fs.time,
        soec.fuel_electrode.ixnodes,
        soec.fuel_electrode.iznodes,
        initialize=0,
        units=pyo.units.K / pyo.units.m
    )

    @soec.fuel_electrode.Constraint(fs.time, soec.fuel_electrode.ixnodes, soec.fuel_electrode.iznodes)
    def dtemperature_dz_eqn(b, t, ix, iz):
        return b.dtemperature_dz[t, ix, iz] == finite_difference(b.temperature, t, ix, iz)

    vars = [soec.dtemperature_z_dz, soec.fuel_electrode.dtemperature_dz]
    cons = [
        soec.dtemperature_z_dz_eqn,
        soec.fuel_electrode.dtemperature_dz_eqn,
    ]
    for var, con in zip(vars, cons):
        for idx, element in var.items():
            iscale.set_scaling_factor(element, 5e-3)
            iscale.constraint_scaling_transform(con[idx], 5e-3)
            
_make_temperature_gradient_terms(m.fs.soec)
def set_indexed_variable_bounds(var, bounds):
    for idx, subvar in var.items():
        subvar.bounds = bounds

set_indexed_variable_bounds(m.fs.soec.soec_module.solid_oxide_cell.fuel_electrode.dtemperature_dz, (-750, 750))

In [ ]:
"""
jac, nlp = iscale.get_jacobian(m, scaled=True)
print("Extreme Jacobian entries:")
for i in iscale.extreme_jacobian_entries(jac=jac, nlp=nlp, large=1000, small=0):
    print(f"    {i[0]:.2e}, [{i[1]}, {i[2]}]")
print("Badly scaled variables:")
for v, sv in iscale.badly_scaled_var_generator(
    m, large=1e3, small=1e-6, zero=1e-12
):
    print(f"    {v} -- {sv} -- {iscale.get_scaling_factor(v)}")
print(f"Jacobian Condition Number: {iscale.jacobian_cond(jac=jac):.2e}")
"""

In [ ]:
# initial solve for design, limited to 100 iterations
res = solver.solve(m, tee=True)

In [ ]:
display_pfd()

In [ ]:
# Print freed decision vars
for v in decision_vars:
    print(f"{v} {pyo.value(v)}")

print("\n")
print("------------------------------------------")
print("Fixed Costs for Optimized Design")
print("------------------------------------------")

tpc = pyo.value(m.fs.costing.total_TPC)  ### the main costing block is now under fs
tasc = pyo.value(m.fs.costing.total_TPC)*1.21*1.093
ac = tasc*0.0707
print(f"TPC = {tpc}")
print(f"TASC = {tasc}")
print(f"Annualized TASC (MM$/yr) = {ac}")

# Parameters
n_op = 8
hourly_rate = 38.50
labor_burden = 30

# Fixed O&M components
annual_op_labor = n_op * hourly_rate * 8760 * (1 + labor_burden/100)/1e6
maint_labor = tpc * 0.4 * 0.019
maint_material = tpc * 0.6 * 0.019
admin_labor = 0.25*(annual_op_labor + maint_labor)
prop_tax_ins = 0.02*tpc
soec_replace = pyo.value(4.2765*m.fs.soec.soec_module.number_cells)/1e6

print("Fixed O&M Costs")
print(f"annual_op_labor (MM$/yr) = {annual_op_labor}")
print(f"maint_labor (MM$/yr) = {maint_labor}")
print(f"maint_material (MM$/yr) = {maint_material}")
print(f"admin_labor (MM$/yr) = {admin_labor}")
print(f"prop_tax_ins (MM$/yr) = {prop_tax_ins}")
print(f"soec_replace (MM$/yr) = {soec_replace}")
total_fixed = annual_op_labor + maint_labor + maint_material + admin_labor + prop_tax_ins + soec_replace
print(f"Annualized Fixed O&M = {total_fixed}")

print("\n")
print("------------------------------------------")
print("SOEC TPC Breakdown")
print("------------------------------------------")

ngcc_soec_costing.display_capital_costs(m) ### added new function to display the costs from power plant costing

In [ ]:
print(total_fixed)
print(ac)

# Check anualized fixed O&M costs
assert total_fixed == pytest.approx(74.690994, rel=1e-3)

# Check anualized TASC
assert ac == pytest.approx(150.09339, rel=1e-3)

In [ ]:
# lock in the design vars
m.fs.soec.soec_module.number_cells.fix()
m.fs.soec.sweep_hx.area.fix()
m.fs.soec.feed_hx01.area.fix()
m.fs.soec.water_heater01.area.fix()
m.fs.soec.water_heater02.area.fix()

# Make sure some constraints imposed by the design optimization
# are respected
@m.fs.Constraint(m.fs.time)
def water_demand_max_constraint(b, t):
    return (
        m.fs.water_demand[t]/100 <= 
        pyo.value(m.fs.water_demand[t]/100)
    )
@m.fs.Constraint(m.fs.time)
def process_water_discharge_max_constraint(b, t):
    return (
        m.fs.process_water_discharge[t]/100 <= 
        pyo.value(m.fs.process_water_discharge[t]/100)
    )
@m.fs.Constraint(m.fs.time)
def aux_load_max_constraint(b, t):
    return (
        m.fs.aux_load[t]/1e3 <= 
        pyo.value(m.fs.aux_load[t]/1e3)
    )
@m.fs.Constraint(m.fs.time)
def CO2_captured_max_constraint(b, t):
    return (
        m.fs.CO2_captured[t]/1e5 <= 
        pyo.value(m.fs.CO2_captured[t]/1e5)
    )

# Deactivate Capital Costing
for blk in m.fs.costing._registered_unit_costing:
    blk.deactivate()
m.fs.costing.deactivate()

# New objective without capital cost.
m.fs.obj_design.deactivate()
m.fs.obj_op = pyo.Objective(
    expr=(
        m.fs.ngcc.total_variable_cost_rate[0] + 
        m.fs.soec.variable_makeup_water_cost[0] +
        (m.fs.ngcc.net_power[0] + m.fs.soec.total_electric_power[0])/1e6 * 100
    )/1e4
)

print(f"Original Objective Value: {pyo.value(m.fs.obj_op)}")
# We should be starting at (or near) the optimal solution from 
# here, so in an effort to save a little time I cut bound_push down
solver.options["bound_push"] = 1e-8
res = solver.solve(m, tee=True)

In [ ]:
"""
m.fs.ngcc.gt.write_pfd(fname="data_pfds/gt_soec_base.svg")
m.fs.ngcc.hrsg.write_pfd(fname="data_pfds/hrsg_soec_base.svg")
m.fs.ngcc.st.write_pfd(fname="data_pfds/st_soec_base.svg")
m.fs.soec.write_pfd(fname="data_pfds/soec_soec_base.svg")
display_pfd()
"""

m.fs.ngcc.gt.streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_gt.csv")
m.fs.ngcc.st.steam_streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_st.csv")
m.fs.ngcc.hrsg.steam_streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_hrsg_steam.csv")
m.fs.ngcc.hrsg.flue_gas_streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_hrsg_gas.csv")
m.fs.soec.streams_dataframe().to_csv("data_tabulated/ngcc_soec_stream_5kg_soec.csv")

### SOEC Performance

In [ ]:
print(f"H2 production rate = {m.fs.soec.tags_output['h2_mass_production']}")
print(f"Water flowrate to SOEC = {m.fs.soec.tags_streams['feed02_F']}")
print(f"Stack conversion = {m.fs.soec.tags_output['water_utilization']}"),
print(f"Overall conversion = {m.fs.soec.tags_output['water_utilization_overall']}")
print(f"Number of cells = {m.fs.soec.tags_output['Number_of_cells']}")
print(f"Cell potential = {m.fs.soec.tags_output['cell_potential']}")
print(f"Current density = {m.fs.soec.tags_output['soec_current_density']}")
print(f"SOEC load (AC) = {m.fs.soec.tags_output['soec_power']}")
print(f"Balance of plant load = {m.fs.soec.tags_output['bop_power']}")
print(f"Natural gas flowrate = {m.fs.ngcc.gt.tags_streams['fuel01_F']}")
print(f"Total load = {m.fs.soec.tags_output['total_electric_power']}")
print(f"NGCC net power = {m.fs.ngcc.tags_output['net_power']}")
print(f"Electric input = {m.fs.soec.tags_output['total_electric_power_per_h2']}")
print(f"GT Power = {m.fs.ngcc.tags_output['gt_power']}")

In [ ]:
assert m.fs.soec.tags_output["total_electric_power_per_h2"].value == pytest.approx(143.996, rel=1e-3)
assert m.fs.soec.tags_output['water_utilization'].value == pytest.approx(56.420884, rel=1e-3)

## Samples

In [ ]:
run_samples = False
df = None
if run_samples:
    # we already ran 477 GT Power and 5 kg/s, so save and we'll load result.
    save_to = f"data/ngcc_soec_{477}_{5000}.json.gz"
    iutil.to_json(m, fname=save_to)
    
    solver.options["max_iter"] = 150
    import pandas as pd
    gt_powers = np.linspace(477., 322., int((477. - 322.)/5.0) + 1).tolist()
    hp_powers = np.linspace(50, 10, 17).tolist()
    
    df = pd.DataFrame(columns=m.fs.tags_output.table_heading())
    i = 1
    save_to_last = None
    for gp in gt_powers:
        gpstr = str(math.ceil(gp))
        m.fs.ngcc.gt.gt_power.fix(-gp*1e6)
        save_to_last_prev = save_to_last
        save_to_last = None
        for hp in [hh/10 for hh in hp_powers]:
            # This is meant to cut off cases that don't produce enough steam to
            # make 5 kg/s h2
            if hp > 5.001 - 0.25*(477.0 - gp)/30:
                continue
            m.fs.soec.hydrogen_product_rate.fix(hp)
            hpstr = str(math.ceil(hp*1000))
            save_to = f"data/ngcc_soec_{gpstr}_{hpstr}.json.gz"
            print(save_to)
            if os.path.exists(save_to):
                iutil.from_json(m, fname=save_to, wts=iutil.StoreSpec(suffix=False))                    
                if save_to_last is None:
                    save_to_last = save_to
            else:
                try:
                    res = solver.solve(m, tee=False)
                except:
                    print("Fail")
                    break
                if not pyo.check_optimal_termination(res):
                    print("Fail")
                    break
                if save_to_last is None:
                    save_to_last = save_to
                iutil.to_json(m, fname=save_to)
            print(f"Overall net power: {-pyo.value(m.fs.ngcc.net_power[0] + m.fs.soec.total_electric_power[0])/1e6} MW")
            #for v in decision_vars:
            #    print(f"{v} {pyo.value(v)}")
            df.loc[i] = m.fs.tags_output.table_row(numeric=True)
            i += 1
        # go to the closest part of the start of the next series.
        try:
            iutil.from_json(m, fname=save_to_last, wts=iutil.StoreSpec(suffix=False))
        except:
            iutil.from_json(m, fname=save_to_last_prev, wts=iutil.StoreSpec(suffix=False))

In [ ]:
if df is not None:
    df.to_csv("data_tabulated/ngcc_soec.csv")